<a href="https://colab.research.google.com/github/Shreyabaral/SocialMediaAnalytics/blob/main/Topic_Modelling_Reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install praw
!pip install pandas nltk gensim pyLDAvis

import praw
import random
import csv

import pandas as pd

!pip install plotly

import pandas as pd
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.0 MB/s eta 0:00:00


In [3]:
# Set up your Reddit instance

client_id = 'pPsx95TX-i0nWFf0_jXygg'
client_secret = 'OSwboGqfZY_-MkgxAjGgGIyjxX_DZA'
user_agent = 'testscript by u/fakebot3'

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

def get_random_posts(subreddit_name, num_posts=500):
    subreddit = reddit.subreddit(subreddit_name)
    posts = list(subreddit.top(limit=500))  # Adjust this limit as needed
    random_posts = random.sample(posts, min(num_posts, len(posts)))

    return [(post.title, post.url) for post in random_posts]

def save_to_csv(posts, filename='random_reddit_posts.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'URL'])  # Header
        writer.writerows(posts)  # Write post data

# Example usage
if __name__ == "__main__":
    subreddit_name = "all" # You can specify any subreddit
    num_posts = 500  # Number of random posts to fetch
    random_posts = get_random_posts(subreddit_name, num_posts)
    save_to_csv(random_posts)

    print(f"Saved {num_posts} random posts to random_reddit_posts.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environment

Saved 500 random posts to random_reddit_posts.csv


In [31]:
df= pd.read_csv('random_reddit_posts.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [19]:
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



(500, 2)

In [20]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,Title,URL
0,*happy bubonic noises*,https://i.redd.it/n7lh6x2wz7841.jpg
1,Wanna hear another joke,https://i.redd.it/00zo3ians0e61.jpg
2,I got off the horse by accident right before a...,https://gfycat.com/HopefulBriefFlickertailsqui...
3,One of the happiest moments in life,https://i.redd.it/rnn14f29gf551.jpg
4,Of course I know him. He is me,https://i.redd.it/kwcqipuz5il51.jpg
...,...,...
495,Mugshots: 5 years for medical marijuana w/expi...,https://i.redd.it/eimfvfmstva51.jpg
496,How would you feel about a feature where if so...,https://www.reddit.com/r/AskReddit/comments/dr...
497,Confusing,https://i.redd.it/2v3ucovw2dk51.jpg
498,Train Simulator is so immersive!,https://media.giphy.com/media/3ov9jLHEBfW5w45p...


In [21]:
# Ensure you have the NLTK stopwords
nltk.download('stopwords')

# Load the data from the CSV file
data = pd.read_csv('random_reddit_posts.csv')

# Preprocessing
stop_words = set(stopwords.words('english'))

def preprocess(text):
    # Simple preprocessing: lowercasing, removing stopwords, and tokenization
    tokens = text.lower().split()
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

# Apply preprocessing to the titles
data['tokens'] = data['Title'].apply(preprocess)

# Create a dictionary and corpus for LDA
dictionary = corpora.Dictionary(data['tokens'])
corpus = [dictionary.doc2bow(tokens) for tokens in data['tokens']]

# Apply LDA
num_topics = 10 # Adjust the number of topics as needed
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic {idx}: {topic}')

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic 0: 0.012*"best" + 0.007*"old" + 0.005*"picture" + 0.005*"lost" + 0.005*"post" + 0.005*"today" + 0.005*"rover" + 0.005*"really" + 0.005*"ordered" + 0.005*"new"
Topic 1: 0.008*"dude" + 0.008*"christmas" + 0.008*"guy" + 0.006*"made" + 0.006*"let" + 0.006*"help" + 0.006*"told" + 0.006*"cat" + 0.006*"never" + 0.006*"son"
Topic 2: 0.012*"day" + 0.012*"everyone" + 0.007*"trump" + 0.005*"years" + 0.005*"dad" + 0.005*"took" + 0.005*"picture" + 0.005*"decided" + 0.005*"die" + 0.005*"really"
Topic 3: 0.015*"years" + 0.010*"got" + 0.008*"boy" + 0.008*"times" + 0.008*"like" + 0.006*"guy" + 0.006*"every" + 0.006*"hong" + 0.006*"many" + 0.006*"maybe"
Topic 4: 0.008*"ever" + 0.008*"cat" + 0.008*"think" + 0.005*"year" + 0.005*"today" + 0.005*"finally" + 0.005*"hard" + 0.005*"world" + 0.005*"joke" + 0.005*"went"
Topic 5: 0.010*"trump" + 0.008*"one" + 0.008*"old" + 0.008*"time" + 0.007*"human" + 0.006*"picture" + 0.006*"photo" + 0.006*"donald" + 0.006*"boys" + 0.006*"us"
Topic 6: 0.012*"man" + 0.00

In [ ]:
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.116407 -0.089436       1        1  13.065958
7      0.136388  0.060746       2        1  12.402235
6     -0.089400  0.091562       3        1  11.650786
5      0.053934 -0.049459       4        1  10.655707
9      0.041580 -0.097306       5        1  10.352741
2     -0.024812  0.027922       6        1   9.331778
3     -0.026815  0.098046       7        1   9.269367
4     -0.020339 -0.029922       8        1   8.987443
1      0.045731 -0.023726       9        1   8.696500
0      0.000140  0.011573      10        1   5.587485, topic_info=         Term      Freq     Total Category  logprob  loglift
280      hong  3.000000  3.000000  Default  30.0000  30.0000
266      like  9.000000  9.000000  Default  29.0000  29.0000
864       dad  7.000000  7.000000  Default  28.0000  28.0000
281      kong  3.000000  3.000000  Default  27.0000  27.0000
0         net  5.000000  5.000000  Default  26.0000  26.0000
...       ...       ...       ...      ...      ...      ...
23     upvote  0.559115  2.994885  Topic10  -5.6473   1.2063
104   picture  0.559114  6.885502  Topic10  -5.6473   0.3738
459     would  0.559114  6.978501  Topic10  -5.6473   0.3604
1202     play  0.559113  3.039312  Topic10  -5.6473   1.1916
635    friend  0.559113  3.078554  Topic10  -5.6473   1.1788

[581 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
228       8  0.814834    100
201       7  0.405620     13
201       9  0.405620     13
58        1  0.384670     15
58        3  0.384670     15
...     ...       ...    ...
101       9  0.074449  years
101      10  0.074449  years
633       3  0.772939   yolo
1517      4  0.793902   york
1407      1  0.507670   zoom

[894 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 7, 6, 10, 3, 4, 5, 2, 1])

In [77]:
import plotly.express as px
import pandas as pd

# Function to get related posts
def display_related_posts(word, data):
    # Filter the DataFrame for related posts
    related_posts = data[data['Title'].str.contains(word, case=False, na=False)]

    # Create a list to hold the URLs
    urls = []
    titles=[]

    # If related posts are found, populate the URLs list
    if not related_posts.empty:
        for _, row in related_posts.iterrows():

            urls.append(row['URL'])
            titles.append(row['Title'])
    else:
        print(f"No related posts found for '{word}'.")

    return urls,titles # Return the list of URLs

# Function to create the interactive word cloud
def create_interactive_word_cloud(lda_model, df):
    words = []
    for idx in range(lda_model.num_topics):
        topic_terms = lda_model.get_topic_terms(idx, topn=30)
        for word_id, prob in topic_terms:
            words.append((dictionary[word_id], prob))

    # Create a DataFrame for the words and their strengths
    df1 = pd.DataFrame(words, columns=['word', 'strength'])
    # Populate related URLs in df1 based on the word and using df
    df1['related_urls'] = df1['word'].apply(lambda x: display_related_posts(x, df)[0])
    df1['titles'] = df1['word'].apply(lambda x: display_related_posts(x, df)[1])
    # Format URLs for hover display
    df1['formatted'] = df1['titles'].apply(lambda title: "<br>".join(title) if title else "No related posts")

    # Create a scatter plot using df1
    fig = px.scatter(
        df1,
        x='word',
        y='strength',
        size='strength',
        hover_name='word',
        hover_data={'formatted': True},  # Show formatted URLs on hover
        title='Interactive Word Cloud'
    )

    # Update the hover template to display URLs nicely
    fig.update_traces(marker=dict(opacity=0.8, line=dict(width=2, color='White')),
        selector=dict(mode='markers'),
        hovertemplate='<b>%{hovertext}</b><br>Related URLs:<br>%{customdata}<extra></extra>',
        hovertext=df1['word'],
        customdata=df1['formatted'],

    )

    # Show the plot
    fig.show()

# Call the function with lda_model and df
create_interactive_word_cloud(lda_model, df)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [78]:
display_related_posts('best', df)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



(['https://i.redd.it/snxd9tay93ky.gif',
  'https://i.redd.it/wcly3hga69k51.jpg',
  'https://i.redd.it/g4vm7de3q4j51.jpg',
  'https://imgur.com/2cfU6dh',
  'https://i.redd.it/nzkdkyfr2hz41.jpg',
  'https://i.redd.it/xyn2xxgal3601.jpg',
  'https://i.redd.it/y8nbh6y019661.jpg',
  'https://i.redd.it/x5wcbq56r0v41.jpg',
  'https://i.redd.it/q7kybd2raw501.jpg'],
 ["This is the best thing I've done on snapchat.",
  'This one goes out to all the Twitch gamers who try their best',
  'She did her best ok?',
  'While showing my 3 year old my Game Boy, I discovered a picture of my best friend from 1999.',
  'Captain Holt is best boi',
  'She took me in 15 years ago and no one makes me as happy. We had the best Xmas. I love my Granma so much.',
  'I drew USA in 3 lines and did my best to seperate states using gaps',
  'CIA really hired the best there is',
  'Flying home for the holidays dressed to surprise the family. Before the flight, I met this stranger at the airport. Instant best friends'])